[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/tabular-classification/documentation-tutorial/tabular-tutorial-part-1.ipynb)



# Welcome to the Openlayer tabular tutorial - Part 1

You should use this notebook together with the [**tabular tutorial**](https://docs.openlayer.com/docs/uploading-your-first-model-and-dataset) from our documentation.

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/tabular-classification/documentation-tutorial/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## 1. Loading the dataset

First, let's import the libraries we need and load the churn training and validation datasets.

In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

We have stored the dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. The churn dataset we use was constructed from the dataset from [this Kaggle competition](https://www.kaggle.com/competitions/churn-modelling/overview).

In [ ]:
TRAINING_SET_URL = "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/Churn+prediction/churn_train.csv"
VALIDATION_SET_URL = "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/Churn+prediction/churn_val.csv"

In [ ]:
# loading and having a look at the training set
training_set = pd.read_csv(TRAINING_SET_URL)
val_set = pd.read_csv(VALIDATION_SET_URL)

training_set.head()

The label we want to learn to predict is in the column `Exited`: retained users have a value of 0 while users that exited have a value of 1. Additionally, we **don't** want to use the `RowNumber`, `CurtomerId`, and `Surname` in our model, so we exclude these columns from our dataset.

In [ ]:
X_train = training_set.iloc[:, 3:-1]
y_train = training_set.iloc[:, -1]

X_val = val_set.iloc[:, 3:-1]
y_val = val_set.iloc[:, -1]

## 2. Pre-processing the data

Notice from one of the previous cell's output that the users' genders and geographies are **categorical features**. Therefore, before feeding the data into the model, we need to encode them. Let's apply a **one-hot-encoding**, which is a common choice when dealing with categorical features.

In [ ]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    for feature, enc in encoders.items():
        print(f"encoding {feature}")
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        df = df.join(enc_df)
        df = df.drop(columns=feature)
    return df

In [ ]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [ ]:
# creating the encoder dict for the categorical features (gender and geography)
encoders = create_encoder_dict(X_train, ['Geography', 'Gender'])

In [ ]:
# encoding the categorical features in our training and validation sets
X_train_one_hot = data_encode_one_hot(X_train, encoders)
X_val_one_hot = data_encode_one_hot(X_val, encoders)

## 3. Training and evaluating our model

We are going to train a gradient boosting classifier on the training data. Let's then check out what the model's performance is in the validation set.

In [ ]:
sklearn_model = GradientBoostingClassifier(random_state=42) 
sklearn_model.fit(X_train_one_hot, y_train)

In [ ]:
print(classification_report(y_val, sklearn_model.predict(X_val_one_hot)))

## 5. Openlayer part!

Now it's up to you! We will just compute a few important variables and concatenate the x and y, because Openlayer expects a single dataframe with features and labels for the upload. 

Head back to the tutorial for an explanation of next few cells.

In [ ]:
feature_names = X_train.columns.values.tolist()
categorical_feature_names = ["Gender", "Geography"]
class_names = ["Retained", "Exited"]

In [ ]:
training_set = pd.concat([X_train, y_train], axis=1)
validation_set = pd.concat([X_val, y_val], axis=1)

In [ ]:
# installing the Openlayer Python API
!pip install openlayer

In [ ]:
# instantiating the client
import openlayer

client = openlayer.OpenlayerClient('YOUR_API_KEY_HERE')

In [ ]:
# creating the project
from openlayer.tasks import TaskType

project = client.create_or_load_project(name="Churn prediction",
                               task_type=TaskType.TabularClassification,
                               description="Evaluation of ML approaches to predict churn")

In [ ]:
# uploading the dataset to the project
dataset = project.add_dataframe(
  df=validation_set,  
  commit_message='churn validation set for October',
  class_names=class_names,  
  label_column_name='Exited',    
  feature_names=feature_names,  
  categorical_feature_names=categorical_feature_names,  
)

In [ ]:
# defining the model's predict probability function
def predict_proba(model, input_features: np.ndarray, col_names: list, one_hot_encoder, encoders):
    # Pre-processing the categorical features
    df = pd.DataFrame(input_features, columns=col_names)
    encoded_df = one_hot_encoder(df, encoders)
    
    # Getting the model's predictions
    preds = model.predict_proba(encoded_df.to_numpy())
    
    return preds

In [ ]:
# uploading the model to the project
from openlayer.models import ModelType

model = project.add_model(
    function=predict_proba, 
    model=sklearn_model,
    model_type=ModelType.sklearn,
    class_names=class_names,
    name='Churn Classifier',
    commit_message='this is my churn classification model',
    feature_names=feature_names,
    train_sample_df=training_set[:3000],
    train_sample_label_column_name='Exited',
    categorical_feature_names=categorical_feature_names,
    requirements_txt_file='requirements.txt',
    col_names=feature_names,
    one_hot_encoder=data_encode_one_hot,
    encoders=encoders,
)